In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

prefix_url='https://graphemy.ir/datasets'
def get_file(name):
    file_name=name if '.csv' in name else f'{name}.csv'
    return f'{prefix_url}/{file_name}'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [3]:
df=pd.read_csv(get_file('teleCust1000t.csv'))
df

,region,tenure,age,marital,address,income,ed,employ,retire,gender,reside,custcat
0,2,13,44,1,9,64.0,4,5,0.0,0,2,1
1,3,11,33,1,7,136.0,5,5,0.0,0,6,4
2,3,68,52,1,24,116.0,1,29,0.0,1,2,3
3,2,33,33,0,12,33.0,2,0,0.0,1,1,1
4,2,23,30,1,9,30.0,1,2,0.0,0,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...
995,3,10,39,0,0,27.0,3,0,0.0,1,3,1
996,1,7,34,0,2,22.0,5,5,0.0,1,1,1
997,3,67,59,0,40,944.0,5,33,0.0,1,1,4
998,3,70,49,0,18,87.0,2,22,0.0,1,1,3


In [4]:
df['custcat'].value_counts()

custcat
3    281
1    266
4    236
2    217
Name: count, dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   region   1000 non-null   int64  
 1   tenure   1000 non-null   int64  
 2   age      1000 non-null   int64  
 3   marital  1000 non-null   int64  
 4   address  1000 non-null   int64  
 5   income   1000 non-null   float64
 6   ed       1000 non-null   int64  
 7   employ   1000 non-null   int64  
 8   retire   1000 non-null   float64
 9   gender   1000 non-null   int64  
 10  reside   1000 non-null   int64  
 11  custcat  1000 non-null   int64  
dtypes: float64(2), int64(10)
memory usage: 93.9 KB


In [6]:
x=df.drop('custcat',axis=1).values
x

array([[ 2., 13., 44., ...,  0.,  0.,  2.],
       [ 3., 11., 33., ...,  0.,  0.,  6.],
       [ 3., 68., 52., ...,  0.,  1.,  2.],
       ...,
       [ 3., 67., 59., ...,  0.,  1.,  1.],
       [ 3., 70., 49., ...,  0.,  1.,  1.],
       [ 3., 50., 36., ...,  0.,  1.,  3.]], shape=(1000, 11))

In [7]:
y=df['custcat'].values


In [8]:
from sklearn.model_selection import train_test_split
x_train , x_test ,y_train , y_test=train_test_split(x,y,test_size=0.2, random_state=42)
x_train.shape ,y_train.shape

((800, 11), (800,))

modeling without data balance


In [9]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model  import LogisticRegression

pip=Pipeline([
    ('scale',StandardScaler()),
    ('model',KNeighborsClassifier()),

])
params=[
    {'model':[KNeighborsClassifier()],
     'model__n_neighbors':[3,5,7],
     'model__p': [1, 2]},
    
    {'model':[DecisionTreeClassifier()],
    'model__max_depth':[3,5,7],
    'model__criterion': ['gini', 'entropy'],         
    'model__min_samples_split': [2, 5, 10],           
    'model__min_samples_leaf': [1, 2, 4]},

    {'model':[LogisticRegression(max_iter=1000)],
    'model__C':[0.1,1,10],
    'model__solver': ['liblinear', 'lbfgs', 'newton-cg', 'sag', 'saga']},

]

In [10]:
gride=GridSearchCV(pip,param_grid=params, cv=5, scoring='accuracy')
gride.fit(x_train,y_train)
print("بهترین مدل:", gride.best_estimator_)
print("بهترین پارامترها:", gride.best_params_)
print("دقت روی داده تست:", gride.score(x_test, y_test))


بهترین مدل: Pipeline(steps=[('scale', StandardScaler()),
                ('model', LogisticRegression(C=10, max_iter=1000))])
بهترین پارامترها: {'model': LogisticRegression(max_iter=1000), 'model__C': 10, 'model__solver': 'lbfgs'}
دقت روی داده تست: 0.435


In [11]:
from sklearn.metrics import classification_report
y_pred=gride.predict(x_test)
print(classification_report(y_test,y_pred))


              precision    recall  f1-score   support

           1       0.46      0.43      0.45        60
           2       0.35      0.33      0.34        39
           3       0.47      0.51      0.49        55
           4       0.43      0.43      0.43        46

    accuracy                           0.43       200
   macro avg       0.43      0.43      0.43       200
weighted avg       0.43      0.43      0.43       200



modeling data balance


In [12]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
x_resampled, y_resampled = smote.fit_resample(x_train, y_train)
x_resampled.shape,y_resampled.shape

((904, 11), (904,))

In [18]:
gride=GridSearchCV(pip,param_grid=params, cv=5, scoring='accuracy')
gride.fit(x_resampled,y_resampled)
print("بهترین مدل:", gride.best_estimator_)
print("بهترین پارامترها:", gride.best_params_)
print("دقت روی داده تست:", gride.score(x_test, y_test))

بهترین مدل: Pipeline(steps=[('scale', StandardScaler()),
                ('model', LogisticRegression(C=0.1, max_iter=1000))])
بهترین پارامترها: {'model': LogisticRegression(max_iter=1000), 'model__C': 0.1, 'model__solver': 'lbfgs'}
دقت روی داده تست: 0.425


In [19]:
from sklearn.metrics import classification_report
y_pred=  gride.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.44      0.42      0.43        60
           2       0.35      0.38      0.37        39
           3       0.49      0.45      0.47        55
           4       0.41      0.43      0.42        46

    accuracy                           0.42       200
   macro avg       0.42      0.42      0.42       200
weighted avg       0.43      0.42      0.43       200

